In [148]:
import re
import pandas as pd

In [149]:
file_path = "variant_3.csv"
df = pd.read_csv(file_path)

df.head()

,Unnamed: 0,index,autographs
0,12,28,"«Истинный вкус состоит…» (в тетр. ПД 833, л. 3..."
1,13,29,"В тетр. ПД 841, л. 128 об. — 128 (черновой пер..."
2,14,19,"В тетр. ПД 831, л. 39 об., 48 об., 49 об., 50 ..."
3,15,25,"В тетр. ПД 845, л. 7 об. — 15, 16 об. — 17 (че..."
4,16,21,"В тетр. ПД 838, л. 16 об. — 17 (черновой ст. 1..."


In [150]:
def expand_doc_ranges_in_text(text):
    """
    Эта функция работает с типом "промежуток".
    Пример: ПД 1044 - ПД 1046
    """
    def repl_full(match):
        a, b = int(match.group(1)), int(match.group(2))
        step = 1 if a <= b else -1
        return '; '.join(f'ПД {i}' for i in range(a, b + step, step))

    text = re.sub(r'ПД\s*(\d+)\s*[–—-]\s*ПД\s*(\d+)', repl_full, text)

    def repl_half(match):
        a, b = int(match.group(1)), int(match.group(2))
        step = 1 if a <= b else -1
        return '; '.join(f'ПД {i}' for i in range(a, b + step, step))

    text = re.sub(r'ПД\s*(\d+)\s*[–—-]\s*(\d+)\b', repl_half, text)

    return text

In [151]:
t_1 = "ПД 8 - ПД 10; ПД 1 - 3; ПД 4 - 2"
res_1 = expand_doc_ranges_in_text(t_1)
print(res_1)

ПД 8; ПД 9; ПД 10; ПД 1; ПД 2; ПД 3; ПД 4; ПД 3; ПД 2


In [152]:
def find_doc_chunks(text):
    """
    Эта функция находит все ПД, при этом разворачивает их, если встретился тип "промежуток".
    """
    text = expand_doc_ranges_in_text(text)

    text_no_parens = re.sub(r'\([^)]*\)', '', text)
    has_pd_outside = bool(re.search(r'\bПД\s*\d+', text_no_parens, flags=re.IGNORECASE))

    if has_pd_outside:
        text_to_search = text_no_parens
    else:
        inside = re.findall(r'\([^)]*\bПД\s*\d+[^)]*\)', text)
        text_to_search = ' '.join([re.sub(r'[()]', '', s) for s in inside]) if inside else text

    pattern = re.compile(r'\b(ПД\s*\d+[^;]*(?:(?!\bПД\s*\d+)[^;])*)', flags=re.IGNORECASE)
    chunks = []
    for m in pattern.finditer(text_to_search):
        chunk = m.group(1).strip()
        chunk = re.sub(r'\s+', ' ', chunk)
        chunks.append((chunk, m.start()))
    return chunks

In [153]:
t_2 = "«Истинный вкус состоит…» (ПД 833; ПД 294)"
res_2 = find_doc_chunks(t_2)
print(res_2)

[('ПД 833', 0), ('ПД 294', 8)]


In [154]:
def expand_range(a, b, start_ob=False, end_ob=False):
    """
    А эта функция преобразует диапазон листов с учётом 'об.' в полный список записей.
    """
    def to_int(s):
        return int(re.sub(r'\D', '', s))
    ai, bi = to_int(a), to_int(b)

    if ai == bi:
        n = ai
        if start_ob and not end_ob:
            return [f"л. {n} об.", f"л. {n}"]
        if end_ob and not start_ob:
            return [f"л. {n}", f"л. {n} об."]
        if start_ob and end_ob:
            return [f"л. {n} об."]
        return [f"л. {n}"]

    step = 1 if ai <= bi else -1
    numbers = list(range(ai, bi + step, step))
    result = []

    if step > 0:
        for i, n in enumerate(numbers):
            is_first = i == 0
            is_last = i == len(numbers) - 1
            if is_first:
                if start_ob:
                    result.append(f"л. {n} об.")
                else:
                    result.append(f"л. {n}")
                    result.append(f"л. {n} об.")
            elif is_last:
                if end_ob:
                    result.append(f"л. {n}")
                    result.append(f"л. {n} об.")
                else:
                    result.append(f"л. {n}")
            else:
                result.append(f"л. {n}")
                result.append(f"л. {n} об.")
    else:
        for i, n in enumerate(numbers):
            is_first = (i == 0)
            is_last = (i == len(numbers) - 1)
            if is_first:
                result.append(f"л. {n}")
            elif is_last:
                if end_ob:
                    result.append(f"л. {n} об.")
                else:
                    result.append(f"л. {n}")
            else:
                result.append(f"л. {n} об.")
                result.append(f"л. {n}")
    return result

In [155]:
t_3 = ("20 об.", "23")
res_3 = expand_range(t_3[0], t_3[1], True, False)
print(res_3)

['л. 20 об.', 'л. 21', 'л. 21 об.', 'л. 22', 'л. 22 об.', 'л. 23']


In [156]:
def process_leaf_group(leaf_group, entries):
    """
    Эта функция находит все листы и разворачивает их.
    Примеры: '44-44 об.', '45', '45-46, 47 об. — 50, 52 об.'
    """
    parts = re.split(r',\s*', leaf_group)
    
    for part in parts:
        part = part.strip()
        if not part:
            continue

        range_match = re.search(r'(\d+\s*(?:об\.)?)\s*[–—-]\s*(\d+\s*(?:об\.)?)', part)
        if range_match:
            a, b = range_match.group(1), range_match.group(2)
            start_ob = a.endswith('об.')
            end_ob = b.endswith('об.')
            a = re.sub(r'\s*об\.$', '', a).strip()
            b = re.sub(r'\s*об\.$', '', b).strip()
            entries.extend(expand_range(a, b, start_ob, end_ob))
            continue

        if part.endswith('об.'):
            num_part = part[:-3].strip()
            num = re.sub(r'\D', '', num_part) 
            if num:
                entries.append(f"л. {num} об.")
            continue

        num = re.sub(r'\D', '', part)
        if num:
            entries.append(f"л. {num}")

In [157]:
t_4 = ("45-46, 47 об. — 50, 52 об.")
res_4 = []
process_leaf_group(t_4, res_4)
print(res_4)

['л. 45', 'л. 45 об.', 'л. 46', 'л. 47 об.', 'л. 48', 'л. 48 об.', 'л. 49', 'л. 49 об.', 'л. 50', 'л. 52 об.']


In [158]:
def extract_leaf_references(text):
    """
    Эта функция собирает все листы после 'л.' из текста, игнорируя скобки.
    """
    s = re.sub(r'\([^)]*\)', '', text)
    s = re.sub(r'\s+', ' ', s).strip()
    
    entries = []
    
    leaf_pattern = re.compile(r'л\.\s*([^;]*)', re.IGNORECASE)
    
    for match in leaf_pattern.finditer(s):
        leaf_group = match.group(1).strip()
        if not leaf_group:
            continue
        
        process_leaf_group(leaf_group, entries)
    
    return entries

In [159]:
t_5 = ("Текст (л. 99) л. 1 об. — 2, л. 5")
res_5 = extract_leaf_references(t_5)
print(res_5)

['л. 1 об.', 'л. 2', 'л. 5']


In [160]:
def extract_from_chunk(chunk):
    """
    Заключительная функция разбирает фрагмент текста с ПД, возвращает нужный вид, который требует лаба.
    """
    results = []

    chunk_no_parens = re.sub(r'\([^)]*\)', '', chunk)
    has_pd_outside = bool(re.search(r'\bПД\s*\d+', chunk_no_parens, flags=re.IGNORECASE))

    if has_pd_outside:
        chunk = re.sub(r'\([^)]*ПД\s*\d+[^)]*\)', '', chunk)
    else:
        inside = re.findall(r'\([^)]*\bПД\s*\d+[^)]*\)', chunk)
        if inside:
            chunk = ' '.join([re.sub(r'[()]', '', s) for s in inside])

    m = re.search(r'ПД\s*(\d+)\s*[–—-]\s*ПД\s*(\d+)', chunk)
    if m:
        start, end = int(m.group(1)), int(m.group(2))
        for pd_num in range(min(start, end), max(start, end) + 1):
            leaves = []
            for i in range(1, 6):
                leaves.append(f"л. {i}")
                leaves.append(f"л. {i} об.")
            results.append((f"ПД {pd_num}", leaves))
        return results

    parts = re.split(r'\bПД\s*', chunk)
    parts = [p.strip() for p in parts if p.strip()]

    for part in parts:
        m = re.match(r'(\d+)(.*)', part)
        if not m:
            continue
        doc_num, rest = m.groups()
        doc_code = f"ПД {doc_num}".strip()
        leaves = extract_leaf_references(rest)
        if not leaves:
            leaves = []
            for i in range(1, 6):
                leaves.append(f"л. {i}")
                leaves.append(f"л. {i} об.")
        results.append((doc_code, leaves))
    return results

In [161]:
t_6 = "ПД 825, беловой, с поправками; ПД 294, л. 9–10"
res_6 = extract_from_chunk(t_6)
print(res_6)

[('ПД 825', ['л. 1', 'л. 1 об.', 'л. 2', 'л. 2 об.', 'л. 3', 'л. 3 об.', 'л. 4', 'л. 4 об.', 'л. 5', 'л. 5 об.']), ('ПД 294', ['л. 9', 'л. 9 об.', 'л. 10'])]


In [162]:
results = []
for _, row in df.iterrows():
    text = str(row.get('autographs', ''))

    text = expand_doc_ranges_in_text(text)
    chunks = find_doc_chunks(text)
    
    flattened = []
    for chunk, _ in chunks:
        doc_entries_list = extract_from_chunk(chunk)
        for doc_code, entries in doc_entries_list:
            for e in entries:
                full_ref = f"{doc_code} {e}".replace("  ", " ").strip()
                flattened.append(full_ref)

    extracted_str = '; '.join(flattened)
    results.append({'index': row.get('index'), 'extracted': extracted_str})

out_df = pd.DataFrame(results)

In [163]:
out_df.to_csv('output.csv')